In [2]:
!pip install langchain-core langgraph>0.2.27

In [4]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

··········


In [5]:
!pip install -qU "langchain[google-genai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [6]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

Enter API key for Google Gemini: ··········


In [8]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Shivang")])

AIMessage(content='Hi Shivang! Nice to meet you.\n\nHow can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--6601dd27-dac0-4503-a068-8cb3aaeaab23-0', usage_metadata={'input_tokens': 8, 'output_tokens': 289, 'total_tokens': 297, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 272}})

In [9]:
# As we can see the model doesn't have any concept of state and doesn't retain our prev conversation

model.invoke([HumanMessage(content = "Who am i?")])

AIMessage(content='That\'s a profound question, and one that humans have contemplated for centuries! As an AI, I can\'t tell you *who you are* in a personal or existential sense, because that\'s something only you can discover and define for yourself.\n\nHowever, I can offer some common lenses through which people explore their identity. When people ask "Who am I?", they often delve into:\n\n1.  **Your Personal History:**\n    *   Your memories, experiences, and the stories that have shaped you.\n    *   The lessons you\'ve learned and the challenges you\'ve overcome.\n\n2.  **Your Personality & Traits:**\n    *   Are you introverted or extroverted? Optimistic or realistic? Creative or analytical?\n    *   What are your core strengths and areas for growth?\n    *   What are your habits and tendencies?\n\n3.  **Your Values & Beliefs:**\n    *   What principles are most important to you (e.g., honesty, compassion, freedom, justice, family, success)?\n    *   What do you believe about the

In [11]:
## We use LangSmith Trace to overcome this. To get around this, we need to pass the entire conversation history into the model.
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Shivang"),
        AIMessage(content="Hello Shivang! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Shivang!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--8292cc66-8131-4736-afbc-fa1451a48309-0', usage_metadata={'input_tokens': 27, 'output_tokens': 50, 'total_tokens': 77, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 44}})

In [12]:
# LangGraph implements a built-in persistence layer, making it ideal for chat applications that support multiple conversational turns.

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# We define a new graph
workflow = StateGraph(state_schema = MessagesState)

# Function that calls the model
def call_model(state: MessagesState):
  response = model.invoke(state["messages"])
  return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [14]:
config = {"configurable": {"thread_id": "abc123"}}


# Why it’s needed?
# Multi-user support: Lets multiple people talk to the bot at once without sharing state.
# Multiple threads per user: A single user could have several different conversation topics running in parallel.
# Resumability: The bot can resume a conversation exactly where it left off if you pass the same thread_id later.

In [15]:
query = "Hi! I'm Shivang."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Shivang! It's great to meet you. How can I help you today?


In [16]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Shivang!


In [17]:
# Now lets try changing our "thread_id" and feed it into our config to act as a different user
config = {"configurable": {"thread_id": "abc234"}}

# query here remains same from prev prompt

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't know your name. As an AI, I don't have access to personal information about you. I don't retain details about who you are from one interaction to the next.


In [18]:
# As expected the chatbot doesn't know us now as we for it are a different user. However, we can always go back to the original conversation (since we are persisting it in a database)

config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Shivang!


In [ ]:
# Now let's try adding some prompt templates

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [20]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [21]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Shivang."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Shivang! A fine name for a landlubber, or perhaps even a future buccaneer!

Welcome aboard, matey! What brings ye to these here waters?


In [22]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Shiver me timbers! Yer name be Shivang, if me ears ain't playin' tricks on me!

Did I get that right, me hearty?


In [23]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [24]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [25]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Shivang."
language = "French"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Bonjour Shivang ! Enchanté de faire votre connaissance.


In [26]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Votre nom est Shivang.


# Managing Conversation History

In [27]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm Shivang"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like going to movies"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 76 + 23"),
    AIMessage(content="99"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm Shivang", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like going to movies', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 76 + 23', additional_kwargs={}, response_metadata={}),
 AIMessage(content='99', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [28]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [29]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't know your name! You haven't told me what it is yet.

Would you like to tell me?


In [30]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked "whats 76 + 23".


In [31]:
# Streaming (not too sure about this rn)

config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

Hi Todd! Here's one for you:

Why don't scientists trust atoms?

Because they make up everything!|